In [287]:
import numpy as np
seed = 128
np.random.seed(seed)
import pandas as pd
from csv import reader
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils

In [321]:
train_data = pd.read_csv("titanic_train.csv", header=0)
train_output = train_data['Survived']
train_data.drop(['Survived','PassengerId', 'Cabin','Name', 'Fare','Ticket','Title'], axis=1, inplace=True)

In [303]:
#train_data['Pclass']=normalise(train_data['Pclass'])
#train_data['Age']=normalise(train_data['Age'])
#train_data['Parch']=normalise(train_data['Parch'])

In [322]:
cat_sex = pd.get_dummies(train_data['Sex'])
train_data['male'] = cat_sex['male']
train_data['female'] = cat_sex['female']
train_data.drop('Sex', axis=1, inplace=True)
train_data.head()

,Pclass,Age,SibSp,Parch,Embarked,male,female
0,3,22,1,0,S,1,0
1,1,38,1,0,C,0,1
2,3,26,0,0,S,0,1
3,1,35,1,0,S,0,1
4,3,35,0,0,S,1,0


In [323]:
cat_embarked = pd.get_dummies(train_data['Embarked'])
train_data['Embarked_C'] = cat_embarked['C']
train_data['Embarked_Q'] = cat_embarked['Q']
train_data['Embarked_S'] = cat_embarked['S']
train_data.drop('Embarked', axis=1, inplace=True)
train_data.head()

,Pclass,Age,SibSp,Parch,male,female,Embarked_C,Embarked_Q,Embarked_S
0,3,22,1,0,1,0,0,0,1
1,1,38,1,0,0,1,1,0,0
2,3,26,0,0,0,1,0,0,1
3,1,35,1,0,0,1,0,0,1
4,3,35,0,0,1,0,0,0,1


In [330]:
nCols = len(train_data.columns)
nCols

9

In [325]:
train_output=pd.get_dummies(train_output)
#train_output

In [361]:
model = Sequential()
model.add(Dense(nCols, input_dim=nCols, kernel_initializer='normal', activation='relu'))
model.add(Dense(2, kernel_initializer='normal', activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [362]:
model.fit(train_data.as_matrix(), train_output.as_matrix(), epochs=800, validation_split=0.3, verbose=False)
print("[INFO] evaluating on testing set...")

(loss, accuracy) = model.evaluate(train_data.as_matrix(), train_output.as_matrix(), verbose=False)

print("[INFO] loss={:.3f}, accuracy: {:.3f}%".format(loss, accuracy * 100))


[INFO] evaluating on testing set...
[INFO] loss=0.406, accuracy: 83.277%


In [296]:
def normalise(column):
    normalised = list()
    normalised = (column-min(column))/(max(column)-min(column))
    return normalised

In [363]:
test_data = pd.read_csv("titanic_test.csv", header=0)
test_data.drop(['PassengerId', 'Cabin','Name', 'Fare','Ticket','Title'], axis=1, inplace=True)
#test_data['Pclass']=normalise(test_data['Pclass'])
#test_data['Age']=normalise(test_data['Age'])
#test_data['Parch']=normalise(test_data['Parch'])
#test_data

In [364]:
cat_sex = pd.get_dummies(test_data['Sex'])
test_data['male'] = cat_sex['male']
test_data['female'] = cat_sex['female']
test_data.drop('Sex', axis=1, inplace=True)
cat_embarked = pd.get_dummies(test_data['Embarked'])
test_data['Embarked_C'] = cat_embarked['C']
test_data['Embarked_Q'] = cat_embarked['Q']
test_data['Embarked_S'] = cat_embarked['S']
test_data.drop('Embarked', axis=1, inplace=True)
test_data.head()

,Pclass,Age,SibSp,Parch,male,female,Embarked_C,Embarked_Q,Embarked_S
0,3,34,0,0,1,0,0,1,0
1,3,47,1,0,0,1,0,0,1
2,2,62,0,0,1,0,0,1,0
3,3,27,0,0,1,0,0,0,1
4,3,22,1,1,0,1,0,0,1


In [365]:
prediction = model.predict(test_data.as_matrix())

In [366]:
prediction = np.argmax(prediction, axis=1) 

In [367]:
passengerID = np.arange(892,1310,1)
survived = prediction
df = pd.DataFrame({'PassengerId':passengerID, 'Survived':survived})
df.to_csv('result_310may2017.csv', index = False)